In [9]:
using DifferentialEquations, FFTW

In [20]:
function solution_lab(ω, F0, α, F1; x0 = [0.3,0.3], ω0=1.0, γ=0.01, η=0.1, cycles=5000, pts_per_cycle=30)
    T = cycles * 2pi / ω
    f(du, u, p, t) = -ω0^2  * u - α*u^3 - γ*du - η*u*u*du + F0*cos(ω*t) + F1*cos(0.95*t)
    problem = SecondOrderODEProblem(f, x0..., [0., T])
    soln = DifferentialEquations.solve(problem, saveat=(1/pts_per_cycle) * 2pi/ω, reltol=1e-8, abstol=1e-8, maxiters=1e8)
end

function Fourier_filter(response; ω_range=[0.5,1.5], ringdown_cycles=10000, pts_per_cycle)
    Δt = response[1][2] - response[1][1]
    
    #k = Int(round(ringdown_time / Δt) + 1)
    k = pts_per_cycle * ringdown_cycles + 1
    
    fft = FFTW.fft(response[2][k:end]) .* Δt # the spectrum
    
    f_axis = FFTW.fftfreq(length(fft), 1/Δt)
    
    f_range = ω_range ./ (2pi)
    
    
    relevant_indices = findall(x -> f_range[1] <= abs(x) <= f_range[2], f_axis)
    fft_filtered = fft[relevant_indices]
    
    parseval_sum = sum((abs.(fft_filtered).^2)) / (length(fft)*Δt)
    sqrt(2 * parseval_sum / (response[1][end] - response[1][k]))
    
end

Fourier_filter (generic function with 1 method)

In [21]:
soln = solution_lab(1.1, 0.01, 1, 1E-5 ,ω0=1.0, γ=0.001, cycles=100000, pts_per_cycle=50)
response = soln.t, getindex.(soln.u, 2);

In [22]:
println("drive: ", Fourier_filter(response, ω_range=[1.095,1.105], pts_per_cycle=50))

drive: 0.5462745916288868


In [23]:
println("at signal: ", Fourier_filter(response, ω_range=[0.945,0.955], pts_per_cycle=50))

at signal: 2.8007884674326032e-5


In [24]:
println("upconverted: ", Fourier_filter(response, ω_range=[1.245,1.255],pts_per_cycle=50))

upconverted: 3.228942983074605e-5


In [25]:
println("up to 3ω: ", Fourier_filter(response, ω_range=[3.25,3.35],pts_per_cycle=50))

up to 3ω: 0.004342837988458274


In [26]:
println("floor: ", Fourier_filter(response, ω_range=[1.485,1.495],pts_per_cycle=50))

floor: 9.197484478632319e-6
